In [ ]:
from __future__ import division

from past.utils import old_div

# Checking path to access other files
try:
    from sippy import *
except ImportError:
    import sys, os

    sys.path.append(os.pardir)
    from sippy import *

import numpy as np
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import matplotlib.pyplot as plt

from scipy.io import savemat

In [ ]:
"""
Created on Fri Jan 19 2018
@author: Giuseppe Armenise, revised by RBdC
In this test, no error occurs. 
Using method='N4SID','MOESP' or 'CVA', if the message
"Kalman filter cannot be calculated" is shown, it means
that the package slycot is not well-installed.
"""



# Example to test SS-methods

# sample time
ts = 1.0

# SISO SS system (n = 2)
A = np.array([[0.89, 0.], [0., 0.45]])
B = np.array([[0.3, 0.7], [2.5, 1.4]])
C = np.array([[0.7, 1.]])
D = np.array([[0.0, 1.0]])

tfin = 500
npts = int(old_div(tfin, ts)) + 1
Time = np.linspace(0, tfin, npts)
m = 2

# Input sequence
U = np.zeros((m, npts))
for i in range(m):
    [U[i, :],_,_] = fset.GBN_seq(npts, 0.05)

##Output
x, yout = fsetSIM.SS_lsim_process_form(A, B, C, D, U)

# measurement noise
noise = fset.white_noise_var(npts, [0.15])

# Output with noise
y_tot = yout + noise

#
# plt.figure(0)
# plt.plot(Time, U[0])
# plt.ylabel("input")
# plt.grid()
# plt.xlabel("Time")
#
plt.figure()
plt.plot(Time, y_tot[0])
plt.ylabel("y_tot")
plt.grid()
plt.xlabel("Time")
plt.title("Ytot")

##System identification
METHOD = ['N4SID']
lege = ['System']
for i in range(len(METHOD)):
    method = METHOD[i]
    sys_id = system_identification(y_tot, U, method, SS_fixed_order = 2 )
    xid, yid = fsetSIM.SS_lsim_process_form(sys_id.A, sys_id.B, sys_id.C, sys_id.D, U, sys_id.x0)
    #
    plt.plot(Time, yid[i])
    lege.append(method) 
    
plt.legend(lege) 

In [ ]:
# Load real data
y = np.load('../data/real-data/d2mice_outputs.npy')
y = 1 - np.nonzero(y)[1][np.newaxis]
inputs = np.load('../data/real-data/d2mice_inputs.npy')

In [ ]:
# Set up system parameters
p = 3
m = 4
k = 30
u = np.hstack((inputs[:,0:2],inputs[:,-2:])).T

# Fit
sys_id = system_identification(y, u, 'N4SID', SS_fixed_order = p, SS_f = k)

In [ ]:
np.set_printoptions(suppress=True, precision=4)
print(sys_id.A)
print(sys_id.B)
print(sys_id.C)
print(sys_id.D)

In [ ]:
plt.figure()
plt.plot(y[0, :], label='true')
# plt.xlim()
xid, yid = fsetSIM.SS_lsim_process_form(sys_id.A, sys_id.B, sys_id.C, sys_id.D, u, sys_id.x0)
plt.plot(yid[0, :], label='sippy');
plt.legend()

In [ ]:
print(y_tot.shape) # Output shape from their example
print(U.shape)     # Input shape from their example

print(y.shape)     # Our data
print(u.shape)

print(yid.shape)   # Shape of the predicted outputs

In [ ]:
np.savez('./fit_params/sippy_gausslds_fit.npz', Ahat=sys_id.A, Bhat=sys_id.B, Chat=sys_id.C, Dhat=sys_id.D)

## Model comparison

In [ ]:
d = np.load('../data/model-comparisons/datasetB/datasetB_large.npz')

# for key in d.keys():
#     print(key)
    
u_train = d['u_train']
y_train = d['y_train']
u_test = d['u_test']
y_test = d['y_test']

A = d['A']

In [ ]:
num_folds = 5
m = u_train.shape[-1]
q = y_train.shape[-1]
p = A.shape[0]

for fold in range(num_folds):
    print(fold)
    u = u_train[fold, :, :].T
    y = y_train[fold, :, :].T
    
#     print(u.shape)
#     print(y.shape)
    sys_id = system_identification(y, u, 'N4SID', SS_fixed_order = p)
    
    savemat('./model-comparison-fits/datasetB_large_gaussian_fold%d.mat' % fold, {'Ahat':sys_id.A, 
             'Bhat':sys_id.B, 'Chat':sys_id.C, 'Dhat':sys_id.D,
             'u_train':u_train, 'u_test':u_test, 'y_train':y_train, 'y_test':y_test,
             'A':d['A'], 'B':d['B'], 'C':d['C'], 'D':d['D'], 'Q':d['Q'], 'N':d['N']})

## Real data

In [ ]:
# Load real data
d = np.load('../data/real-data/baseline-performance/train_test_info.npz', allow_pickle=True)

for key in d.keys():
    print(key)
    
y_train = d['y_train']
y_test = d['y_test']
u_train = d['u_train']
u_test = d['u_test']
sessions_train = d['sessions_train']
sessions_test = d['sessions_test']
trials_test = d['trials_test']
seeds = d['seeds']

p = 3
m = 4
k = 30
q = 1
nfolds = 5
# y = 1 - np.nonzero(y)[1][np.newaxis]
# inputs = np.load('../data/real-data/d2mice_inputs.npy')

In [ ]:
A = np.zeros((nfolds, p, p))
B = np.zeros((nfolds, p, m))
C = np.zeros((nfolds, q, p))
D = np.zeros((nfolds, q, m))
Q = np.zeros((nfolds, p, p))

for fold in range(5):
    u = u_train[fold].T
    y = y_train[fold].T
    sys_id = system_identification(y, u, 'N4SID', SS_fixed_order = p, SS_f = k)
    
    A[fold, :, :] = sys_id.A
    B[fold, :, :] = sys_id.B
    C[fold, :, :] = sys_id.C
    D[fold, :, :] = sys_id.D
    Q[fold, :, :] = sys_id.Q
    
param_dict = {'A' : A, 'B' : B, 'C' : C, 'D' : D, 'Q' : Q, 'y_train' : y_train, 'y_test' : y_test,
              'u_train' : u_train, 'u_test' : u_test, 'sessions_train' : sessions_train, 
              'sessions_test' : sessions_test, 'trials_test' : trials_test, 'seeds' : seeds}
savemat('../data/real-data/baseline-performance/gaussian_train_params.mat', param_dict)
    

## EM

In [ ]:
import scipy.io as sio

for i in range(7):
    # Load real data
    d = np.load('../data/em-inits/datasetD_q1/best-lds-%d.npz' % i)
    y = d['y']
    u = d['u']
    p = d['A'].shape[0]
    k = p
    
    sys_id = system_identification(y, u, 'N4SID', SS_fixed_order = p, SS_f = k)
    np.savez('../data/em-inits/datasetD_q1/gaussian-%d.npz' % i, Ahat=sys_id.A, Bhat=sys_id.B, 
             Chat=sys_id.C, Dhat=sys_id.D, Qhat=sys_id.Q, seed=d['seed'], A=d['A'], B=d['B'], C=d['C'], 
             C_new=d['C_new'], D=d['D'], Q=d['Q'], Q0=d['Q0'], x0=d['x0'], muu=d['muu'], 
             Qu=d['Qu'], y=y, z=d['z'], u=u, x=d['x'])
    
    data_dict = {}
    for key in d.keys():
        data_dict[key] = d[key]
        
    data_dict['Ahat'] = sys_id.A
    data_dict['Bhat'] = sys_id.B
    data_dict['Chat'] = sys_id.C
    data_dict['Dhat'] = sys_id.D
    data_dict['Qhat'] = sys_id.Q
    
#     sio.savemat('data/em-inits/datasetD_q1/gaussian-%d.mat' % i, data_dict)